In [ ]:
import torch

print(f'{torch.version.git_version=}')
print(f'{torch.version.hip=}')
print(f'{torch.version.debug=}')
print(f'{torch.version.cuda=}')

cpu = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU {torch.cuda.current_device()}')
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type(torch.DoubleTensor)
    


In [ ]:
from itertools import product
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
import tdg

import tdg.HMC as HMC
import tdg.plot as visualize

import h5py as h5

# Target

Let us try to pick a system with some physical relevance by compare against some data in [2212.05177](https://arxiv.org/abs/2212.05177).

In [ ]:
# See eq. (7.6)

def alpha3(alpha):
    
    log4 = torch.log(torch.tensor(4.))
    return 0.25 * alpha**2 * (
                    1
                    + (1.5 - log4) * alpha
                    + 3*(0.16079 - (log4-1))*alpha**2
                    # + O(alpha^3)
    )

# alpha3 = torch.tensor([
#     # Some points of interpolation for the curve, if need be.
#     # Extracted from their Figure 12.
#     [0.598902, 0.074818],[0.511862, 0.057783],[0.444455, 0.045412],[0.361013, 0.031059],
#     [0.305007, 0.022658],[0.250488, 0.015285],[0.177552, 0.007921],[0.125091, 0.003866],
#     [0.056770, 0.000702],[0.000282, -0.000084],[-0.061888, 0.000928],[-0.112767, 0.003178],
#     [-0.183703, 0.008094],[-0.243079, 0.013748],[-0.281439, 0.018160],[-0.315505, 0.022290],
#     [-0.355172, 0.027626],[-0.401856, 0.034315],[-0.435591, 0.038935],[-0.482788, 0.045873],
#     [-0.526242, 0.052000],[-0.568489, 0.057922],[-0.614529, 0.063739],[-0.651008, 0.067718],])

MCDataPositive = torch.tensor([[0.561347, 0.070021],[0.473085, 0.051458],[0.382930, 0.034817],
                               [0.291280, 0.020615],[0.195786, 0.009559],[0.098755, 0.002458],])

MCDataNegative = torch.tensor([[-0.622725, 0.067143],[-0.490006, 0.047254],[-0.395163, 0.032882],
                               [-0.309663, 0.019606],[-0.255783, 0.014222],[-0.234334, 0.012622],])

In [ ]:
def comparison_plot(ax):
    x = torch.linspace(-0.65,+0.65,1000)
    ax.plot(x.cpu(), alpha3(x).cpu(), color='black')
    # ±5% for the cutoff dependence.
    ax.fill_between(x.cpu(), alpha3(0.95*x).cpu(), alpha3(1.05*x).cpu(), color='gray', alpha=0.2)
    ax.plot(MCDataNegative[:,0].cpu(), MCDataNegative[:,1].cpu(), color='gray', marker='o', linestyle='none')
    ax.plot(MCDataPositive[:,0].cpu(), MCDataPositive[:,1].cpu(), color='blue', marker='v', linestyle='none')
    ax.set_xlim([-0.65,+0.65])
    ax.set_ylim([0,0.075])    

fig, ax = plt.subplots(1,1, figsize=(10,6))
comparison_plot(ax)

# Tuning

First, to establish their notation:

$$\begin{align}
    \alpha(x) &= -\left(\log x a\right)^{-1} 
    &
    \alpha    &= \alpha(k_F) \text{ if no argument is provided.}
\end{align}$$

We should aim for $\alpha>0$ so that we do not need to compute the gap $M \Delta^2/4\pi$ (5.1).  Their results are good up to $\alpha \lesssim 0.6$ or so.

What does that correspond to in terms of the number density
$$ \rho = N/L^2 = \frac{g}{4\pi} k_F^2 ?$$

Massaging the definition of $\alpha$ one finds
$$\begin{align}
    e^{-1/\alpha} = k_F a = \sqrt{\frac{4\pi N}{gL^2}} a = \sqrt{\frac{N}{g\pi}} \frac{2\pi a}{L}
\end{align}$$

Since for our system $g=2$,
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{N}{2\pi}} \frac{2\pi a}{L} = \sqrt{\frac{N}{2\pi}} \tilde{a}
\end{align}$$

Suppose we want to stay in the dilute regime $\text{sparsity} = N/\Lambda\lesssim 0.1 $ to keep spatial discretization errors at bay.  Then we should solve
$$\begin{align}
    e^{-1/\alpha} = \sqrt{\frac{\text{sparsity} \Lambda}{2\pi}} \tilde{a}
\end{align}$$
for $\tilde{a}$ to find
$$\begin{align}
    \tilde{a} = \frac{e^{-1/\alpha}}{\sqrt{\frac{\Lambda \text{sparsity}}{2\pi}}}
\end{align}$$

In [ ]:
def target_ere(alpha, lattice, sparsity):
    
    atilde = torch.exp(-1/alpha) / torch.sqrt( lattice.sites * sparsity / (2*torch.pi))
    
    return tdg.EffectiveRangeExpansion(torch.tensor((atilde,)))

They show the contact density [their (7.5)] for a range of α in Fig. 12.  Something to note is that the energy density in their definition [their (5.1)] requires subtracting off the gap.

We can access the attractive channel with α < 0.  

In [ ]:
alpha_target = torch.tensor(-0.3)
Lattice = tdg.Lattice(7)
sparsity = torch.tensor(0.1) # so we should tune µ until we find about 5 particles
Z = tdg.Luescher.Zeta2D()

We can tune to the continuum limit.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))

# First draw the zeta function.
exact = torch.linspace(-5.001, 30.001, 1000)
Z.plot(ax, exact, color='gray')

# and the analytic piece of the ERE
ere = target_ere(alpha_target, Lattice, sparsity)
analytic = ere.analytic(exact)
ax.plot(exact.cpu(), analytic.cpu().clone().detach(), color='black', linestyle='dashed')

def plot(ax, nx, alpha, starting_lattice, starting_sparsity):
    Lattice = tdg.Lattice(nx)
    # Take the continuum limit holding the number of particles fixed
    # and increasing the number of sites.
    sparsity = starting_sparsity*(starting_lattice.nx/nx)**2
    
    ere = target_ere(alpha, Lattice, sparsity)
    tuning = tdg.AnalyticTuning(ere, Lattice)
    A1 = tdg.ReducedTwoBodyA1Hamiltonian(Lattice, [tdg.LegoSphere(r) for r in tuning.radii])
    E  = A1.eigenenergies(tuning.C)
    x  = E * Lattice.sites / (2*torch.pi)**2
    z  = Z(x) / torch.pi**2
    ax.plot(
        x.cpu().detach().numpy(),
        z.cpu().detach().numpy(),
        linestyle='none', marker='o',
        label=f'nx={Lattice.nx}'
        )

for nx in range(7,25,2):
    plot(ax, nx, alpha_target, Lattice, sparsity)

ax.legend()
ax.set_xlim((min(exact).cpu(), max(exact).cpu()))
ax.set_ylim((-0,5))

# Parameter Scan

Let us check performance as we scale towards the temporal and spatial continuum limits, holding the physics fixed.

In [ ]:
NT = (8, 16, 24, 32, 48, 64,)# 96, 128)
NX = (7, 9, 11, 13, 15, 17, 19, 21)

storage = './scaling.h5'
configurations = 1000

def key(nx, nt, root='/'):
    return f'{root}/{nx=}/{nt=}'

# Heavy Compute!

In [ ]:
mu = 0.
beta = 3

In [ ]:
for nx in NX:
    
    tuning = tdg.AnalyticTuning(ere, tdg.Lattice(nx))

    for nt in NT:
        
        # Chop out expensive corner
        if nx > 15 and nt > 48:
            continue
              
        try:
            with h5.File(storage, 'r') as f:
                generation_time = f[f'{key(nx, nt)}/generation_time'][()]
                print(f'{nx=} {nt=} {generation_time}')
                continue
        except:
            
            S = tuning.Action(nt, beta, torch.tensor(mu))
            print(S)

            H = HMC.Hamiltonian(S)
            integrator = HMC.Omelyan(H, 30, 1)
            hmc = HMC.MarkovChain(H, integrator)

            t0 = time.time()

            ensemble = tdg.ensemble.GrandCanonical(S).generate(configurations, hmc, start='hot')
            
            t1 = time.time()
            ensemble.generation_time = t1 - t0

            print(f'{nx=} {nt=} {ensemble.generation_time}')
            
            with h5.File(storage, 'a') as f:
                ensemble.to_h5(f.create_group(key(nx,nt)))
                

# Read in generation times

In [ ]:
times = dict()

for (nx, nt) in product(NX, NT):
    try:
        with h5.File(storage, 'r') as f:
            times[(nx, nt)] = f[f'{key(nx,nt)}/generation_time'][()]
    except:
        pass

df = pd.DataFrame(((nx, nt, time) for (nx, nt), time in times.items()), columns=('nx', 'nt', 'time'))

# Plot

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(16,12))

for (nx, group) in df.groupby('nx'):
    ax[0,0].plot(group['nt'], group['time']/configurations, marker='o', label=f'{nx=}')
    ax[1,0].plot(group['nt'], group['time']/group['nt']/configurations, marker='o', label=f'{nx=}')
    
ax[0,0].set_title(f'Seconds for 1 HMC trajectory with 30 MD steps')
ax[0,0].set_xlabel('nt')
ax[0,0].set_ylabel('time [seconds]')
ax[0,0].legend()

ax[1,0].set_title(f'Seconds Per Timeslice for 1 HMC trajectory with 30 MD steps')
ax[1,0].set_xlabel('nt')
ax[1,0].set_ylabel('time [seconds] / nt')
ax[1,0].legend()

    
for (nt, group) in df.groupby('nt'):
    ax[0,1].plot(group['nx']**2, group['time']/configurations, marker='o', label=f'{nt=}')
    ax[1,1].plot(group['nx']**2, group['time']/group['nx']**2/configurations, marker='o', label=f'{nt=}')
    
    
ax[0,1].set_title(f'Seconds for 1 HMC trajectory with 30 MD steps')
ax[0,1].set_xlabel('nx^2')
ax[0,1].set_ylabel('time [seconds]')
ax[0,1].legend()

ax[1,1].set_title(f'Seconds Per Spatial Volume for 1 HMC trajectory with 30 MD steps')
ax[1,1].set_xlabel('nx^2')
ax[1,1].set_ylabel('time [seconds] / nx^2')
ax[1,1].legend()